## A Guide to Digitizing Historical Maps Using QGIS, Python, and Lightroom

Authors: Haicheng Xu & Idaliya Grigoryeva

### Intro

This guide will walk you through the complete process of digitizing a historical map, using a map of Indonesia from our project as an example. Our goal is to digitize the road network in Indonesia to create a friction surface that represents the travel cost between any two points. With this objective in mind, we focused on finding a map where the roads are clearly distinguishable from other features. To achieve accurate digitization, we selected a map where only the roads are red, making them easy to extract.

Steps for digitizing: 
1. Eliminate the saturation for all colors except for red in Lightroom
2. Use road_to_1 function below to convert the processed map from step 1 to black and white
3. Import the black and white map from step 2 into QGIS
4. Convert the raster to points then connect the points using nearest points to path in QGIS


### Map Download

Link to the Indonesia map: https://digitalcollections.universiteitleiden.nl/view/item/814037?solr_nav%5Bid%5D=425f4cdcd85c99e9fcfd&solr_nav%5Bpage%5D=0&solr_nav%5Boffset%5D=11 

Note: Please download the "Original Master" version of the map for the best result.